# Demo for PyTorch Mask RCNN
Base Repo:  https://github.com/multimodallearning/pytorch-mask-rcnn

Sample Repo:  https://github.com/michhar/pytorch-mask-rcnn-samples

**Note:  Run this notebook inside of the Sample Repo after cloning**

## Prerequisites

1.  PyTorch (tested with version below) - **note**:  further testing required as PyTorch has updated to 1.0 which is in preview (as of this notebook update) and the code was originally using 0.4.  It's likely the model class and training script will need to be updated as well.  Troubleshooting is happening for the custom extension builds as well. (2018-09-28)

## Check Setup

In [ ]:
import sys
sys.prefix

In [ ]:
import torch
torch.__version__

### Download the Model

Here:  https://drive.google.com/open?id=1VV6WgX_RNl6a9Yi9-Pe7ZyVKHRJZSKkm

And upload to the root directory of the repo on this machine (where this notebook lives).

## Setup and Run Inference

In [ ]:
# Am I running 32 or 64 bit Python
import struct
print(struct.calcsize("P") * 8)

In [ ]:
%load_ext autoreload
%autoreload
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob
from io import BytesIO
import requests
from PIL import Image
%matplotlib inline

import coco
# from pycocotools import coco
import utils
import model as modellib
import visualize

import torch
# import pycocotools

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to trained weights file
# Download this file and place in the root of your
# project (See README file for details)
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "models", "mask_rcnn_coco.pth")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    # GPU_COUNT = 0 for CPU
    GPU_COUNT = 0
    IMAGES_PER_GPU = 1
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.pth")


config = InferenceConfig()
config.display()

# Create model object.
model = modellib.MaskRCNN(model_dir=MODEL_DIR, config=config)
model = model.to(device=device)

In [ ]:
# Load weights trained on MS-COCO
print(COCO_MODEL_PATH)
model.load_state_dict(torch.load(COCO_MODEL_PATH))

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# Load a random image from the images folder
# file_names = glob.glob(os.path.join('images', '*.jpg'))
# image = skimage.io.imread(os.path.join(random.choice(file_names)))

# Or load file from the internet
req = requests.get('https://cdn.pixabay.com/photo/2015/06/20/13/55/man-815795__340.jpg')
image = np.asarray(Image.open(BytesIO(req.content)))

In [ ]:
# Run detection
results = model.detect([image])

In [ ]:
# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                            class_names, r['scores'])
plt.show()

## Run an actual training experiment

In [ ]:
# ! python coco.py train --dataset=mask_rcnn_coco.pth --model=imagenet

In [ ]:
%%writefile demo.py
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob
from io import BytesIO
import requests
from PIL import Image

# import coco
from pycocotools import coco
import utils
import model as modellib
import visualize

import torch
import pycocotools

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to trained weights file
# Download this file and place in the root of your
# project (See README file for details)
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "models", "mask_rcnn_coco.pth")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    # GPU_COUNT = 0 for CPU
    GPU_COUNT = 0
    IMAGES_PER_GPU = 1
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.pth")


config = InferenceConfig()
config.display()

# Create model object.
model = modellib.MaskRCNN(model_dir=MODEL_DIR, config=config)
model = model.to(device=device)

# Load weights trained on MS-COCO
print(COCO_MODEL_PATH)
model.load_state_dict(torch.load(COCO_MODEL_PATH))

# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# Load a random image from the images folder
# file_names = glob.glob(os.path.join('images', '*.jpg'))
# image = skimage.io.imread(os.path.join(random.choice(file_names)))

# Or load file from the internet
req = requests.get('https://cdn.pixabay.com/photo/2015/06/20/13/55/man-815795__340.jpg')
image = np.asarray(Image.open(BytesIO(req.content)))

# Run detection
results = model.detect([image])

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                            class_names, r['scores'])
plt.show()